In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
import re
import json
from tqdm import tqdm 
# URL endpoint của Wikidata
ENDPOINT_URL = "https://query.wikidata.org/sparql"

# Đọc dữ liệu từ file JSON
data = []
for filename in ["test.json", "train.json"]:
    with open(filename, "r", encoding="utf-8") as json_file:
        data.extend(json.load(json_file))

# Cập nhật danh sách truy vấn từ dữ liệu JSON
queries = [
    {
        "query": item["sparql"],
        "target_variable": item["target_variable"].lstrip("?")  # Loại bỏ dấu "?" khỏi target_variable
    }
    for item in data
]
# Hàm chạy truy vấn SPARQL
def run_query(query):
    sparql = SPARQLWrapper(ENDPOINT_URL)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    try:
        return sparql.query().convert()
    except Exception as e:
        print(f"Lỗi khi chạy truy vấn: {query}\n{e}")
        return {"results": {"bindings": []}}

# Hàm xác định tất cả các biến trong câu truy vấn
def extract_variables(query):
    return re.findall(r'\?(\w+)', query)

# Hàm tách các bộ ba từ câu truy vấn
def extract_triples(query):
    pattern = re.compile(r'\{(.*?)\}', re.DOTALL)
    match = pattern.search(query)
    if match:
        triples = match.group(1).split(" . ")
        return [triple.strip() for triple in triples if triple.strip()]
    return []

# File output RDF (N-Triples)
output_file = "wikidata_dump.nt"

with open(output_file, "w", encoding="utf-8") as f:
    for index, query_info in tqdm(enumerate(queries), total=len(queries)): 
        # if index % 100 == 0:  # Kiểm tra mỗi 100 câu truy vấn
        #     print(f"Đã duyệt qua {index} câu truy vấn.")
        triplet_set = set()
        query = query_info["query"]
        target_variable = query_info["target_variable"]
        triples = extract_triples(query)
        variables = extract_variables(query)
        variables = list(set(variables) - {target_variable})  # Giữ lại các biến khác nhau, loại bỏ biến mục tiêu  # Loại bỏ biến mục tiêu khỏi danh sách biến trung gian
        # Tạo từ điển lưu giá trị của các biến
        variable_values = {var: [] for var in variables}
        
        # Truy vấn từng biến trung gian trước
        for var in variables:
            for triple in triples:
                if f"?{var}" in triple and f"{target_variable}" not in triple:
                    var_query = f"SELECT ?{var} WHERE {{ {triple} }}"
                    results = run_query(var_query)
                    variable_values[var].extend([res[var]["value"] for res in results["results"]["bindings"] if var in res])
        # Truy vấn biến mục tiêu cuối cùng
        results = run_query(query)
        for result in results["results"]["bindings"]:
            target_value = result.get(target_variable, {}).get("value", "")
            target_uri = f"<{target_value}>" if target_value else ""
            
            # Lưu bộ ba RDF hoàn chỉnh
            for triple in triples:
                modified_triple = triple
                
                # Thay thế tất cả các biến trung gian
                for var, values in variable_values.items():
                    for value in values:
                        modified_triple = modified_triple.replace(f"?{var}", f"<{value}>")
                # Thay thế biến mục tiêu
                modified_triple = modified_triple.replace(f"?{target_variable}", target_uri)
                # Chuẩn hóa cú pháp RDF
                modified_triple = modified_triple.replace("wd:", "<http://www.wikidata.org/entity/")
                modified_triple = modified_triple.replace("wdt:", "<http://www.wikidata.org/prop/direct/")
                parts = modified_triple.split()  # Tách các thành phần
                for i in range(len(parts)):
                    if parts[i] == ".":
                        continue  # Bỏ qua trường hợp dấu chấm
                    if not parts[i].startswith("<") and not parts[i].endswith(">"):
                        parts[i] = f"<{parts[i].strip()}>"
                    elif parts[i].startswith("<") and not parts[i].endswith(">"):
                        parts[i] = parts[i].strip() + ">"
                    elif not parts[i].startswith("<") and parts[i].endswith(">"):
                        parts[i] = "<" + parts[i].strip()
                modified_triple = " ".join(parts)  # Kết hợp lại thành chuỗi
                modified_triple = modified_triple + ".\n"
                triplet_set.add(modified_triple)
        for triplet in triplet_set:
            f.write(triplet)

print(f"✅ File dump RDF đã được tạo thành công tại {output_file}!")


100%|██████████| 8282/8282 [2:59:10<00:00,  1.30s/it]  

✅ File dump RDF đã được tạo thành công tại wikidata_dump.nt!
